<a href="https://colab.research.google.com/github/MeenalShah13/SentimentAnalysisCOVID/blob/main/Sentiment_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing of tweets

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
tweets=pd.read_csv('covid19_tweets.csv')
tweets.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [ ]:
df = tweets

In [ ]:
df.shape

(179108, 13)

In [ ]:
df.isnull().sum() 

user_name               0
user_location       36771
user_description    10286
user_created            0
user_followers          0
user_friends            0
user_favourites         0
user_verified           0
date                    0
text                    0
hashtags            51334
source                 77
is_retweet              0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.duplicated().sum()

0

In [ ]:
df = df.drop_duplicates(subset=None, keep='first', inplace=False)

In [ ]:
df.isnull().sum()

user_name           0
user_location       0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
is_retweet          0
dtype: int64

In [ ]:
df.shape

(99138, 13)

## Sampling

In [ ]:
sample = df.sample(n=15000)
#sample.to_csv('sample_data.csv',index=False)

In [ ]:
sample.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
127508,راشد,Kingdom of Saudi Arabia,آغتراب ﻻفقدت إلي تحب كن مافوقك سماء وكن ماتحتك...,2019-05-18 03:54:26,18,212,52,False,2020-08-14 05:27:36,The Saudi Health Organization released #Tabaud...,"['TabaudApp', 'COVID19']",Twitter Web App,False
50461,kim kartazian,Civic Center Tenderloin,Liberal. Punching up for the Tenderloin's most...,2018-07-11 18:01:04,408,353,8571,False,2020-07-31 17:28:44,@SF_DPH finally testing our mostly #Latinx bui...,['Latinx'],Twitter for iPhone,False
157654,ShanghaiEye,"上海, 中华人民共和国","News and stories with a Shanghai perspective, ...",2015-10-16 06:24:19,29456,515,488,True,2020-08-22 08:36:46,...including stipulating that masks must be wo...,['COVID19'],Twitter Web App,False
43757,AMC,"Tucson, AZ",Old liberal cat with kitten's heart. Mom/Mimi....,2010-03-21 00:07:35,12561,12393,154851,False,2020-07-30 20:15:53,"#HermanCain wasn’t sick at #TrumpTulsaRally, w...","['HermanCain', 'TrumpTulsaRally']",Twitter for iPhone,False
125406,Trap Suit Podcast,"Los Angeles, CA",Official Twitter of the Trap Suit Podcast 🎥🎙📺 ...,2020-05-11 02:50:00,71,223,329,False,2020-08-14 07:18:27,167K deaths due to #covid19 https://t.co/HJKbn...,['covid19'],Twitter for iPhone,False


In [ ]:
df1 = sample.drop(['user_created','user_description','user_followers','user_friends','user_favourites','user_verified','source','is_retweet'],axis=1)
df1.head()

,user_name,user_location,date,text,hashtags
127508,راشد,Kingdom of Saudi Arabia,2020-08-14 05:27:36,The Saudi Health Organization released #Tabaud...,"['TabaudApp', 'COVID19']"
50461,kim kartazian,Civic Center Tenderloin,2020-07-31 17:28:44,@SF_DPH finally testing our mostly #Latinx bui...,['Latinx']
157654,ShanghaiEye,"上海, 中华人民共和国",2020-08-22 08:36:46,...including stipulating that masks must be wo...,['COVID19']
43757,AMC,"Tucson, AZ",2020-07-30 20:15:53,"#HermanCain wasn’t sick at #TrumpTulsaRally, w...","['HermanCain', 'TrumpTulsaRally']"
125406,Trap Suit Podcast,"Los Angeles, CA",2020-08-14 07:18:27,167K deaths due to #covid19 https://t.co/HJKbn...,['covid19']


In [ ]:
df1['user_location'].value_counts()

India                          442
United States                  254
New Delhi, India               206
Mumbai, India                  187
Washington, DC                 125
                              ... 
Mississippi                      1
Sioux Falls, SD                  1
Nairobi Metropolitan, Kenya      1
Newfangled                       1
Yorkshire Leeds UK               1
Name: user_location, Length: 5555, dtype: int64

In [ ]:
df1.shape

(15000, 5)

In [ ]:
df1.to_csv('sampledata.csv',index=False)

## Sentiment Polarity


In [ ]:
df1 = pd.read_csv("sampledata.csv")

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
blanks = []  # start with an empty list

for i,un,ul,d,t,h in df1.itertuples():  # iterate over the DataFrame
    if type(t)==str:            # avoid NaN values
        if t.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

df1.drop(blanks, inplace=True)
df1.shape

(15000, 5)

In [ ]:
df1['scores'] = df1['text'].apply(lambda review: sid.polarity_scores(review))

df1['compound']  = df1['scores'].apply(lambda score_dict: score_dict['compound'])


df1['label'] = df1['compound'].apply(lambda c: 'pos' if c >0 else ('neu' if c==0 else 'neg'))


df1.head()

,user_name,user_location,date,text,hashtags,scores,compound,label
127508,راشد,Kingdom of Saudi Arabia,2020-08-14 05:27:36,The Saudi Health Organization released #Tabaud...,"['TabaudApp', 'COVID19']","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
50461,kim kartazian,Civic Center Tenderloin,2020-07-31 17:28:44,@SF_DPH finally testing our mostly #Latinx bui...,['Latinx'],"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.3182,pos
157654,ShanghaiEye,"上海, 中华人民共和国",2020-08-22 08:36:46,...including stipulating that masks must be wo...,['COVID19'],"{'neg': 0.118, 'neu': 0.699, 'pos': 0.183, 'co...",0.2960,pos
43757,AMC,"Tucson, AZ",2020-07-30 20:15:53,"#HermanCain wasn’t sick at #TrumpTulsaRally, w...","['HermanCain', 'TrumpTulsaRally']","{'neg': 0.138, 'neu': 0.711, 'pos': 0.151, 'co...",0.0772,pos
125406,Trap Suit Podcast,"Los Angeles, CA",2020-08-14 07:18:27,167K deaths due to #covid19 https://t.co/HJKbn...,['covid19'],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu


In [ ]:
df1.to_csv('finalsample.csv',index=False)

In [ ]:
df1['label'].value_counts()

pos    5602
neu    5547
neg    3851
Name: label, dtype: int64

## Contractions removal

In [ ]:
import re
contractions_dict = {"can`t":"can not",
                     "won`t":"will not",
                     "don`t":"do not",
                     "aren`t":"are not",
                     "i`d":"i would",
                     "couldn`t": "could not",
                     "shouldn`t": "should not",
                     "wouldn`t": "would not",
                     "isn`t": "is not",
                     "it`s": "it is",
                     "didn`t": "did not",
                     "weren`t": "were not",
                     "mustn`t": "must not",
                    }

def prepare_data(df1:pd.DataFrame) -> pd.DataFrame:
    
    df1["text"] = df1["text"] \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))
        
    df1["label"] = df1["label"].map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )
    return df1.text.values, df1.label.values
def replace_words(string:str, dictionary:dict):
    for k, v in dictionary.items():
        string = string.replace(k, v)
    return string


## Random forest classifier

In [ ]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

X = df1.text
Y =df1.label

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



X_train = X_train \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))

X_test = X_test \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))

y_train =  y_train.map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )
y_test = y_test.map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )


(12000,) (12000,)
(3000,) (3000,)


In [ ]:
train_tweets, train_labels = X_train,y_train
test_tweets, test_labels = X_test,y_test

In [ ]:
pip install keras


_______________________________________

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tweets)
train_tokenized = tokenizer.texts_to_matrix(
                             train_tweets,
                             mode='tfidf'
                             )
test_tokenized = tokenizer.texts_to_matrix(
                             test_tweets,
                             mode='tfidf'
                             )

In [ ]:
from  sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(
                                n_estimators=500, 
                                min_samples_leaf=2,
                                oob_score=True,
                                n_jobs=-1,
                                )
forest.fit(train_tokenized,train_labels)
print(f"Train score: {forest.score(train_tokenized,train_labels)}")
print(f"OOB score: {forest.oob_score_}")

Train score: 0.8465
OOB score: 0.7280833333333333


In [ ]:
print("Test score:", forest.score(test_tokenized,test_labels))

Test score: 0.7236666666666667
